This notebook uses below given notebooks to make predictions.

1. Pretrain Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain
2. Finetune Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune
3. Inference Notebook: https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference
4. Roberta + SVM: this notebook

In [ ]:
# !ls ../input/autonlp-finetune-save-weights/model0.bin

In [ ]:
import os
import gc
import sys
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import (
    Dataset, DataLoader, 
    SequentialSampler, RandomSampler
)

from transformers import (
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup
)
from transformers import (AutoModel, AutoTokenizer, 
                          AutoModelForSequenceClassification)
from transformers import AdamW


import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff


from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [ ]:
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
from sklearn import model_selection
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.KFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for f, (t_, v_) in enumerate(kf.split(X=data)):
        data.loc[v_, 'kfold'] = f
    return data
train_data = create_folds(train_data, num_splits=5)
test_data = create_folds(test_data, num_splits=5)

In [ ]:

num_bins = 12
train_data.loc[:,'bins'] = pd.cut(train_data['target'],bins=num_bins,labels=False)

target = train_data['target'].to_numpy()
# bins = train_data.kfold.to_numpy()
bins = train_data.bins.to_numpy()

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
# model_name = '../input/robertalarge'

In [ ]:
configuration = {
    'batch_size':16,
    'max_len':256,
    'nfolds':5,
    'seed':21,
    'model_name': '../input/k/sujithkpanoor/required-data/renjith/',
}

def seed_everything(seed=21):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=configuration['seed'])

In [ ]:
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.excerpt = df['excerpt'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
#                                 return_token_type_ids=True,
                                return_tensors='pt',
                                max_length=configuration['max_len'],
                                padding='max_length',
                                truncation=True)
        return encode
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.roberta = AutoModel.from_pretrained(configuration['model_name'])    
        self.head = AttentionHead(1024,1024,1)
        self.linear = nn.Linear(1024,1)
        self.dropout = nn.Dropout(0.2)
#         self.linear2 = nn.Linear(512,1)

    def forward(self,**xb):
        x = self.roberta(**xb)[0]
        x = self.head(x)
        x = self.linear(x)
        x = self.dropout(x)
#         x = self.linear2(x)
        return x

In [ ]:
from transformers import AutoConfig
def make_model(model_name=configuration['model_name'], num_labels=1):
    tokenizer = AutoTokenizer.from_pretrained(configuration['model_name'])
    config = AutoConfig.from_pretrained(model_name)
    config.update({'num_labels':num_labels})
    model = CommonLitModel(model_name, config=config)
    return model, tokenizer

def make_optimizer(model, optimizer_name="AdamW"):
    optimizer_grouped_parameters = get_optimizer_params(model)
    kwargs = {
            'lr':5e-5,
            'weight_decay':0.01,
            # 'betas': (0.9, 0.98),
            # 'eps': 1e-06
    }
    if optimizer_name == "LAMB":
        optimizer = Lamb(optimizer_grouped_parameters, **kwargs)
        return optimizer
    elif optimizer_name == "Adam":
        from torch.optim import Adam
        optimizer = Adam(optimizer_grouped_parameters, **kwargs)
        return optimizer
    elif optimizer_name == "AdamW":
        optimizer = AdamW(optimizer_grouped_parameters, **kwargs)
        return optimizer
    else:
        raise Exception('Unknown optimizer: {}'.format(optimizer_name))

def make_scheduler(optimizer, decay_name='linear', t_max=None, warmup_steps=None):
    if decay_name == 'step':
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=[30, 60, 90],
            gamma=0.1
        )
    elif decay_name == 'cosine':
        scheduler = lrs.CosineAnnealingLR(
            optimizer,
            T_max=t_max
        )
    elif decay_name == "cosine_warmup":
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_max
        )
    elif decay_name == "linear":
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=warmup_steps, 
            num_training_steps=t_max
        )
    else:
        raise Exception('Unknown lr scheduler: {}'.format(decay_type))    
    return scheduler    

def make_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
    fold=0
):
    train_set, valid_set = data[data['kfold']!=fold], data[data['kfold']==fold]
    train_dataset = DatasetRetriever(data, tokenizer, max_len)
    valid_dataset = DatasetRetriever(valid_set, tokenizer, max_len)

#     train_sampler = RandomSampler(train_dataset)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
#         sampler=train_sampler, 
        pin_memory=True, 
        drop_last=False, 
        num_workers=4
    )

#     valid_sampler = SequentialSampler(valid_dataset)
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size=batch_size // 2, 
#         sampler=valid_sampler, 
        pin_memory=True, 
        drop_last=False, 
        num_workers=4
    )

    return train_loader, valid_loader

In [ ]:
def get_optimizer_params(model):
    # differential learning rate and weight decay
    param_optimizer = list(model.named_parameters())
    learning_rate = 5e-5
    no_decay = ['bias', 'gamma', 'beta']
    group1=['layer.0.','layer.1.','layer.2.','layer.3.']
    group2=['layer.4.','layer.5.','layer.6.','layer.7.']    
    group3=['layer.8.','layer.9.','layer.10.','layer.11.']
    group_all=['layer.0.','layer.1.','layer.2.','layer.3.','layer.4.','layer.5.','layer.6.','layer.7.','layer.8.','layer.9.','layer.10.','layer.11.']
    optimizer_parameters = [
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': 0.01},
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': 0.01, 'lr': learning_rate/2.6},
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': 0.01, 'lr': learning_rate},
        {'params': [p for n, p in model.roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': 0.01, 'lr': learning_rate*2.6},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': 0.0},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': 0.0, 'lr': learning_rate/2.6},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': 0.0, 'lr': learning_rate},
        {'params': [p for n, p in model.roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': 0.0, 'lr': learning_rate*2.6},
        {'params': [p for n, p in model.named_parameters() if "roberta" not in n], 'lr':1e-3, "momentum" : 0.99},
    ]
    return optimizer_parameters

In [ ]:
class CommonLitModel(nn.Module):
    def __init__(
        self, 
        model_name, 
        config,  
        multisample_dropout=False,
        output_hidden_states=False
    ):
        super(CommonLitModel, self).__init__()
        self.config = config
        self.roberta = AutoModel.from_pretrained(
            model_name, 
            output_hidden_states=output_hidden_states
        )
        self.head = AttentionHead(1024,1024,1)
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        if multisample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(0.5) for _ in range(5)
            ])
        else:
            self.dropouts = nn.ModuleList([nn.Dropout(0.3)])
        #self.regressor = nn.Linear(config.hidden_size*2, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
        self._init_weights(self.layer_norm)
        self._init_weights(self.regressor)
 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
 
    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[1]
        sequence_output = self.layer_norm(sequence_output)
 
        # max-avg head
        # average_pool = torch.mean(sequence_output, 1)
        # max_pool, _ = torch.max(sequence_output, 1)
        # concat_sequence_output = torch.cat((average_pool, max_pool), 1)
 
        # multi-sample dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.regressor(dropout(sequence_output))
            else:
                logits += self.regressor(dropout(sequence_output))
        
        logits /= len(self.dropouts)
 
        # calculate loss
        loss = None
        if labels is not None:
            # regression task
            loss_fn = torch.nn.MSELoss()
            logits = logits.view(-1).to(labels.dtype)
            loss = torch.sqrt(loss_fn(logits, labels.view(-1)))
        
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [ ]:
def convert_examples_to_features(data, tokenizer, max_len, is_test=False):
    data = data.replace('\n', '')
    tok = tokenizer.encode_plus(
        data, 
        max_length=max_len, 
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    curr_sent = {}
    padding_length = max_len - len(tok['input_ids'])
    curr_sent['input_ids'] = tok['input_ids'] + ([tokenizer.pad_token_id] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        if 'excerpt' in self.data.columns:
            self.excerpts = self.data.excerpt.values.tolist()
        else:
            self.excerpts = self.data.text.values.tolist()
#         self.targets = self.data.target.values.tolist()
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
#         excerpt, label = self.excerpts[item], self.targets[item]
        excerpt = self.excerpts[item]
        features = convert_examples_to_features(
            excerpt, self.tokenizer, 
            self.max_len, self.is_test
        )
        return {
            'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
            'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
            'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
#             'label':torch.tensor(label, dtype=torch.double),
        }

In [ ]:
class Evaluator:
    def __init__(self, model, scalar=None):
        self.model = model
        self.scalar = scalar

    def evaluate(self, data_loader, tokenizer):
        preds = []
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch_idx, batch_data in enumerate(data_loader):
                input_ids, attention_mask, token_type_ids = batch_data['input_ids'], \
                    batch_data['attention_mask'], batch_data['token_type_ids']
                input_ids, attention_mask, token_type_ids = input_ids.cuda(), \
                    attention_mask.cuda(), token_type_ids.cuda()
                
                if self.scalar is not None:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                        )
                else:
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids
                    )
                
                logits = outputs[0].detach().cpu().numpy().squeeze().tolist()
                preds += logits
        return preds

In [ ]:
def config(fold,model_name):
    torch.manual_seed(2021)
    torch.cuda.manual_seed(2021)
    torch.cuda.manual_seed_all(2021)
    epochs = 8
    max_len = 256
    batch_size = 16

    model, tokenizer = make_model(model_name=model_name, num_labels=1)
    train_loader, valid_loader = make_loader(
        train_data, tokenizer, max_len=max_len,
        batch_size=batch_size, fold=fold
    )

    import math
    num_update_steps_per_epoch = len(train_loader)
    max_train_steps = epochs * num_update_steps_per_epoch
    warmup_proportion = 0
    if warmup_proportion != 0:
        warmup_steps = math.ceil((max_train_steps * 2) / 100)
    else:
        warmup_steps = 0

    optimizer = make_optimizer(model, "AdamW")
    scheduler = make_scheduler(
        optimizer, decay_name='cosine_warmup', 
        t_max=max_train_steps, 
        warmup_steps=warmup_steps
    )    

    if torch.cuda.device_count() >= 1:
        print('Model pushed to {} GPU(s), type {}.'.format(
            torch.cuda.device_count(), 
            torch.cuda.get_device_name(0))
        )
        model = model.cuda() 
    else:
        raise ValueError('CPU training is not supported')

    # scaler = torch.cuda.amp.GradScaler()
    scaler = None

    result_dict = {
        'epoch':[], 
        'train_loss': [], 
        'val_loss' : [], 
        'best_val_loss': np.inf
    }
    return (
        model, tokenizer, 
        optimizer, scheduler, 
        scaler, train_loader, 
        valid_loader, result_dict, 
        epochs
    )

In [ ]:
def make_test_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
):
    
    test_dataset = DatasetRetriever(data, tokenizer, max_len, is_test=True)
    test_sampler = SequentialSampler(test_dataset)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size // 2, 
        sampler=test_sampler, 
        pin_memory=False, 
        drop_last=False, 
        num_workers=0
    )

    return test_loader

In [ ]:
def run(fold=0, model_name=None, path=None):
#     model, tokenizer, test_loader, scaler = config(fold, model_name, load_model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    
    model, tokenizer, optimizer, scheduler, scaler, train_loader, valid_loader, result_dict, epochs = config(fold,model_name)    
    model.load_state_dict(torch.load(path))
    model.to(device)  
    
    test_loader = make_test_loader(
        test_data, tokenizer, max_len=configuration['max_len'],
        batch_size=configuration['batch_size']
    )
    
    import time

    evaluator = Evaluator(model, scaler)

    test_time_list = []

    torch.cuda.synchronize()
    tic1 = time.time()

    preds = evaluator.evaluate(test_loader, tokenizer)

    torch.cuda.synchronize()
    tic2 = time.time() 
    test_time_list.append(tic2 - tic1)
    
    del model, tokenizer, test_loader, scaler
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

In [ ]:
# !ls ../input/autonlp-finetune-save-weights/


In [ ]:
%%time
pred_df1 = pd.DataFrame()
pred_df2 = pd.DataFrame()
# pred_df3 = pd.DataFrame()
# pred_df4 = pd.DataFrame()
# pred_df5 = pd.DataFrame()
# pred_df6 = pd.DataFrame()
for fold in tqdm(range(5)):
    pred_df1[f'fold{fold}'] = run(fold, configuration['model_name'], '../input/autonlptwoweights/model0.bin')
    pred_df2[f'fold{fold+5}'] = run(fold, configuration['model_name'], '../input/autonlptwoweights/model1.bin')
#     pred_df3[f'fold{fold+5}'] = run(fold, configuration['model_name'], '../input/autonlp-finetune-save-weights/model0.bin')
#     pred_df4[f'fold{fold+10}'] = run(fold, configuration['model_name'], '../input/autonlp-finetune-save-weights/model0.bin')
#     pred_df5[f'fold{fold+15}'] = run(fold, configuration['model_name'], '../input/autonlp-finetune-save-weights/model0.bin')
#     pred_df6[f'fold{fold+20}'] = run(fold, configuration['model_name'], '../input/autonlp-finetune-save-weights/model0.bin')

In [ ]:
# res =  (pred_df1.mean(axis=1) + pred_df3.mean(axis=1) + pred_df4.mean(axis=1) + pred_df5.mean(axis=1) + pred_df6.mean(axis=1))/5
# res

In [ ]:

# def get_embeddings(df,path,fold,model_name,plot_losses=True, verbose=True):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"{device} is used")
            
# #     train_loader, valid_loader = make_loader(
# #         df, tokenizer, max_len=256,
# #         batch_size=16, fold=fold
# #     )
#     model, tokenizer, optimizer, scheduler, scaler, train_loader, valid_loader, result_dict, epochs = config(fold,model_name)    
#     model.load_state_dict(torch.load(path))
#     model.to(device)
#     model.eval()
    
# #     tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    
# #     ds = CLRPDataset(df,tokenizer)
#     ds = DatasetRetriever(df, tokenizer, 256)
#     dl = DataLoader(ds,
#                   batch_size = 128,
#                   shuffle=False,
#                   num_workers = 4,
#                   pin_memory=True,
#                   drop_last=False
#                  )
        
#     embeddings = list()
#     with torch.no_grad():
#         for i, batch_data in enumerate(dl):
#             input_ids, attention_mask, token_type_ids = batch_data['input_ids'], batch_data['attention_mask'], batch_data['token_type_ids']
#             input_ids, attention_mask, token_type_ids = input_ids.cuda(), attention_mask.cuda(), token_type_ids.cuda()
# #             inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in batch_data.items()}
#             outputs = model(
#                         input_ids=input_ids,
#                         attention_mask=attention_mask,
#                         token_type_ids=token_type_ids
# #                         labels=labels
#             )
# #             outputs = list(model(**inputs))[0]
# #             outputs = outputs.detach().cpu().numpy()
# #             print(outputs[0])
#             embeddings.extend(outputs[0])
# #             print(outputs[0])
# #             print("======")
# #             print(outputs[1])
# #             embeddings.extend(outputs[1])

#     return (np.array(embeddings)).reshape(-1, 1)

In [ ]:
# t1 = '../input/autonlptwoweights/model0.bin'
# train_embeddings1 =  get_embeddings(train_data, t1, 0, configuration['model_name'])
# test_embeddings1 = get_embeddings(test_data, t1, 0, configuration['model_name'])

# t2 = '../input/autonlptwoweights/model1.bin'
# train_embeddings2 =  get_embeddings(train_data, t2, 1, configuration['model_name'])
# test_embeddings2 = get_embeddings(test_data, t2, 1, configuration['model_name'])

# t3 = '../input/robertabasesaveweights/model2.bin'
# train_embeddings3 =  get_embeddings(train_data, t3,2,'../input/roberta-base')
# test_embeddings3 = get_embeddings(test_data, t3,2, '../input/roberta-base')

# t4 = '../input/robertabasesaveweights/model3.bin'
# train_embeddings4 =  get_embeddings(train_data, t4,3, '../input/roberta-base')
# test_embeddings4 = get_embeddings(test_data, t4,3, '../input/roberta-base')

# t5 = '../input/robertabasesaveweights/model4.bin'
# train_embeddings5 =  get_embeddings(train_data, t5,4, '../input/roberta-base')
# test_embeddings5 = get_embeddings(test_data, t5,4, '../input/roberta-base')

## svm

In [ ]:
# def get_preds_svm(X,y,X_test,bins=bins,nfolds=5,C=10,kernel='rbf'):
#     scores = list()
#     preds = np.zeros((X_test.shape[0]))
    
#     kfolds = StratifiedKFold(n_splits=5, random_state=configuration['seed'], shuffle=True)
#     for k, (train_idx,valid_idx) in enumerate(kfolds.split(X,bins)):
# #     for fold in tqdm(range(5), total=5):
# #         train_loader, val_loader = make_loader(train_data, tokenizer, 256, 16, fold=fold)
#         model = SVR(C=C,kernel=kernel,gamma='auto')
#         X_train,y_train = X[train_idx], y[train_idx]
#         X_valid,y_valid = X[valid_idx], y[valid_idx]
        
#         model.fit(X_train,y_train)
#         prediction = model.predict(X_valid)
#         score = rmse_score(prediction,y_valid)
#         print(f'Fold {k} , rmse score: {score}')
#         scores.append(score)
#         preds += model.predict(X_test)
        
#     print("mean rmse",np.mean(scores))
#     return np.array(preds)/nfolds

In [ ]:
# 0.22552367492969502

In [ ]:
# svm_preds1 = get_preds_svm(train_embeddings1,target,test_embeddings1)
# svm_preds2 = get_preds_svm(train_embeddings2,target,test_embeddings2)
# svm_preds3 = get_preds_svm(train_embeddings3,target,test_embeddings3)
# svm_preds4 = get_preds_svm(train_embeddings4,target,test_embeddings4)
# svm_preds5 = get_preds_svm(train_embeddings5,target,test_embeddings5)

In [ ]:
# pred_df2

In [ ]:
# p = pred_df2 + pred_df1
# p

In [ ]:
svm = list((pred_df1['fold0'] + pred_df2['fold5'] )/2)

In [ ]:
# svm = (pred_df1.mean(axis=1) + pred_df2.mean(axis=1))/2

In [ ]:
# sample['svm_preds1'] = svm_preds1
# sample['svm_preds2'] = svm_preds2
# # # sample['svm_preds3'] = svm_preds3
# # # sample['svm_preds4'] = svm_preds4
# # # sample['svm_preds5'] = svm_preds5
# sample['pred_df1'] = pred_df1.mean(axis=1)
# sample['pred_df2'] = pred_df2.mean(axis=1)
# # sample['pred_df3'] = pred_df3.mean(axis=1)
# # sample['pred_df4'] = pred_df4.mean(axis=1)
# # sample['pred_df5'] = pred_df5.mean(axis=1)
# # sample['pred_df6'] = pred_df6.mean(axis=1)
# # sample.to_csv('test.csv',index=False)
# sample

In [ ]:
# result = (svm_preds1 + svm_preds2 + svm_preds3 + svm_preds4 + svm_preds5 + pred_df1.mean(axis=1) + pred_df3.mean(axis=1) + pred_df4.mean(axis=1) + pred_df5.mean(axis=1) + pred_df6.mean(axis=1))/10

In [ ]:
# svm_preds = (svm_preds1 + svm_preds2 + svm_preds3 + svm_preds4 + svm_preds5)/5

In [ ]:
sample.target = svm
sample.to_csv('submission.csv',index=False)
sample